# import files

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
import scipy.io as spio
import os
import matplotlib.pyplot as plt
import numpy as np

Using TensorFlow backend.
/usr/local/Cellar/python3/3.6.0/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [112]:
train_x = spio.loadmat("MSdata.mat")['trainx']
test_x = spio.loadmat('MSdata.mat')['testx']
train_y = spio.loadmat('MSdata.mat')['trainy']

# preprocess data

In [113]:
train_no = 400000
train_data = np.concatenate((train_x, train_y), axis = 1)
np.random.shuffle(train_data)

In [114]:

train_music = train_data[:train_no,:90]
train_year = train_data[:train_no,90:]
test_music = train_data[train_no:,:90]
test_year = train_data[train_no:,90:]

music_max = np.max(train_music, axis = 0).reshape(1,train_music.shape[1])
music_min = train_music.min(axis = 0).reshape(1,train_music.shape[1])
year_max = train_year.max()
year_min = train_year.min()

#train_music = train_music-train_music.max
# train_year = 
# test_music = 
# test_year = 

print(train_year)

[[ 2007.]
 [ 1984.]
 [ 1997.]
 ..., 
 [ 2008.]
 [ 1994.]
 [ 1997.]]


In [115]:
print(music_max.shape)
train_music = (train_music - music_min)/(music_max-music_min)
test_music = (test_music - music_min)/(music_max-music_min)
test_x = (test_x - music_min)/(music_max-music_min)
train_year = (train_year-year_min+50)/(year_max-year_min+100)
# test_year = (test_year-year_min+50)/(year_max-year_min+100)
print(train_year)

(1, 90)
[[ 0.71428571]
 [ 0.59259259]
 [ 0.66137566]
 ..., 
 [ 0.71957672]
 [ 0.64550265]
 [ 0.66137566]]


In [124]:
nn = Sequential()
nn.add(Dense(500,input_shape = train_music[0].shape,
       activation = 'relu'))
nn.add(Dense(50,input_shape = train_music[0].shape,
       activation = 'relu'))
nn.add(Dense(200, activation = 'relu'))
nn.add(Dense(1, activation = 'relu'))
nn.compile(optimizer='rmsprop', loss='mean_squared_error')
# sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
# nn.compile(optimizer = sgd,loss = 'mean_squared_error')
nn.fit(train_music, train_year, epochs = 30, batch_size = 50)



Epoch 1/30
400000/400000 [==============================] - 19s 47us/step - loss: 0.0037
Epoch 2/30
400000/400000 [==============================] - 19s 47us/step - loss: 0.0028
Epoch 3/30
400000/400000 [==============================] - 19s 47us/step - loss: 0.0027
Epoch 4/30
400000/400000 [==============================] - 19s 47us/step - loss: 0.0026
Epoch 5/30
400000/400000 [==============================] - 19s 47us/step - loss: 0.0026
Epoch 6/30
400000/400000 [==============================] - 19s 47us/step - loss: 0.0026
Epoch 7/30
400000/400000 [==============================] - 19s 47us/step - loss: 0.0026
Epoch 8/30
400000/400000 [==============================] - 18s 44us/step - loss: 0.0026
Epoch 9/30
400000/400000 [==============================] - 18s 45us/step - loss: 0.0025
Epoch 10/30
400000/400000 [==============================] - 18s 45us/step - loss: 0.0025
Epoch 11/30
400000/400000 [==============================] - 18s 46us/step - loss: 0.0025
Epoch 12/30
400000/

In [80]:
nn.evaluate(x=test_music, y=test_year, batch_size=None, verbose=1, sample_weight=None)

ValueError: Input arrays should have the same number of samples as target arrays. Found 51630 input samples and 63715 target samples.

In [138]:
pred = (nn.predict(test_x)*(year_max-year_min+100))+year_min-50

In [139]:
pred2 = pred.round()
print(pred2)
no = np.arange(1,pred2.shape[0]+1,1).reshape(pred2.shape[0],1)
print(pred2.shape,np.arange(1,pred2.shape[0],1).shape)
pred2 = np.concatenate((no, pred2),axis = 1)
print(pred2.shape)
np.savetxt("pred1.csv", pred2, fmt='%i', delimiter = ",")
print(pred2)
print(test_year)

[[ 1993.]
 [ 2005.]
 [ 2003.]
 ..., 
 [ 2004.]
 [ 2004.]
 [ 2003.]]
(51630, 1) (51629,)
(51630, 2)
[[  1.00000000e+00   1.99300000e+03]
 [  2.00000000e+00   2.00500000e+03]
 [  3.00000000e+00   2.00300000e+03]
 ..., 
 [  5.16280000e+04   2.00400000e+03]
 [  5.16290000e+04   2.00400000e+03]
 [  5.16300000e+04   2.00300000e+03]]
[[ 2009.]
 [ 1997.]
 [ 2005.]
 ..., 
 [ 2004.]
 [ 1981.]
 [ 2007.]]


In [104]:
print(np.average(np.abs(pred.round()-test_year)))

6.41031154359


In [84]:
music_max = train_music.max(axis = 1)
music_min = train_music.min(axis = 1)
year_max = train_year.max()
year_min = train_year.min()
print(music_max)
print(year_max)
print(music_min)
print(year_min)

[  896.79382  1951.73956  3049.05491 ...,  2766.17086  1916.92536
   513.89883]
2011.0
[-402.58749 -584.83798 -729.85652 ..., -607.39863 -597.22494 -236.20379]
1922.0


In [136]:
a = np.ones((5,5))
print(a)

[[ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]]


In [137]:
print(np.linalg.eigvals(a))
print(np.linalg.eig(a))

[  5.00000000e+00   5.65333918e-49   0.00000000e+00   0.00000000e+00
   1.73014109e-64]
(array([  5.00000000e+00,   5.65333918e-49,   0.00000000e+00,
         0.00000000e+00,   1.73014109e-64]), array([[ -4.47213595e-01,   1.13586253e-16,   1.13586253e-16,
          6.83745817e-32,  -1.11062157e-16],
       [ -4.47213595e-01,   8.66025404e-01,   8.66025404e-01,
          5.21314185e-16,  -8.46780726e-01],
       [ -4.47213595e-01,  -2.88675135e-01,  -2.88675135e-01,
         -6.24089760e-17,   1.11088069e-01],
       [ -4.47213595e-01,  -2.88675135e-01,  -2.88675135e-01,
         -7.07106781e-01,   3.67846328e-01],
       [ -4.47213595e-01,  -2.88675135e-01,  -2.88675135e-01,
          7.07106781e-01,   3.67846328e-01]]))
